In [ ]:
from scipy import optimize

## Renewable energy sector

In [30]:
class RenewableEnergySector:

    @staticmethod
    def output(capital, alpha, tfp, **params):
        """Renewable energy output."""
        return tfp * capital**alpha

    @classmethod
    def revenue(cls, capital, alpha, energy_price, tfp, **params):
        """Renewable energy revenue."""
        return cls.subsidy(energy_price) * cls.output(capital, alpha, tfp, **params)
    
    @staticmethod
    def subsidy(energy_price, **params):
        """For now assume that subsidy is the energy price."""
        return energy_price

    @staticmethod
    def total_costs(capital, gross_interest_rate, **params):
        """Renewable energy production costs."""
        return gross_interest_rate * capital

    @classmethod
    def profits(cls, capital, energy_price, alpha, gross_interest_rate, tfp, **params):
        """Renewable energy sector profits."""
        pi = (cls.revenue(capital, alpha, energy_price, tfp, **params) - 
              cls.total_costs(capital, gross_interest_rate, **params))
        return pi
    
    @classmethod
    def capital_demand(cls, alpha, energy_price, gross_interest_rate, tfp, **params):
        """
        Renewable energy sector demand for physical capital.
        
        Note
        ----
        The demand for physical capital is a function of the price of energy
        (which may or may not be subsidized) and model parameters.
        
        """
        return (alpha * tfp * (cls.subsidy(energy_price) / gross_interest_rate))**(1 / (1 - alpha))

    

## Non-renewable energy sector

In [32]:
class NonRenewableEnergySector:

    @staticmethod
    def capital_adjustment_costs(capital, previous_capital, phi):
        """Convex capital adjustment cost function."""
        return (phi / 2) * (capital - previous_capital)**2
    
    @staticmethod
    def marginal_adjustment_costs(capital, previous_capital, phi):
        """Marginal costs of adjusting stock of capital."""
        return phi * (capital - previous_capital)
    
    @classmethod
    def total_cost_capital(cls, capital, previous_capital, phi):
        """Total costs of capital for use in producing energy from fossil fuels."""
        return gross_interest_rate * capital + cls.adjustment_costs(capital, previous_capital, phi)
    
    @classmethod
    def total_cost_fossil_fuel(cls, fossil_fuel, fossil_fuel_price):
        """Total costs of purchasing fossil fuel for use in energy production."""
        return fossil_fuel_price * fossil_fuel
    
    @classmethod
    def total_costs(cls, capital, previous_capital, fossil_fuel, fossil_fuel_price, phi):
        """Total costs of producing energy from fossil fuels."""
        costs = (cls.total_cost_capital(capital, previous_capital, phi) +
                 cls.total_cost_fossil_fuel(fossil_fuel, fossil_fuel_price))
        return costs
    
    @classmethod
    def marginal_cost_capital(cls, capital, previous_capital, phi):
        """Costs of adjusting stock of physical capital."""
        return gross_interest_rate + cls.marginal_adjustment_costs(capital, previous_capital, phi)
    
    @staticmethod
    def output(capital, fossil_fuel, beta, tfp, **params):
        """Non-renewable energy output."""
        return tfp * capital**beta * fossil_fuel**(1 - beta)
    
    @classmethod
    def revenue(cls, capital, fossil_fuel, beta, energy_price, tfp, **params):
        """Renewable energy revenue."""
        return energy_price * cls.output(capital, fossil_fuel, beta, tfp, **params)
    
    @classmethod
    def capital_output_ratio(cls, capital, fossil_fuel, beta, tfp, **params):
        """Ratio of capital stock to output."""
        return capital / cls.output(tfp, capital, fossil_fuel, beta)
    
    @classmethod
    def value_marginal_product_capital(cls, capital, fossil_fuel, beta, energy_price, tfp, **params):
        """Contribution to firm revenue of the marginal unit of installed capital."""
        return energy_price * beta / cls.capital_output_ratio(capital, fossil_fuel, beta, tfp, **params)
    
    @classmethod
    def net_value_marginal_product_capital(capital, previous_capital, fossil_fuel, beta, energy_price, phi, tfp, **params):
        return cls.value_marginal_product_capital(capital, fossil_fuel, beta, energy_price, tfp, **params) - cls.marginal_cost_capital(capital, previous_capital, phi)

    @classmethod
    def future_capital_demand(cls, capital, previous_capital, fossil_fuel, beta, energy_price, eta, phi, tfp, **params):
        """Demand for capital is forward looking?"""
        return capital + (cls.net_value_marginal_product_capital(capital, previous_capital, fossil_fuel, beta, energy_price, phi, tfp, **params) / (eta * phi))
        
    @staticmethod
    def fossil_fuel_demand(beta, energy_price, fossil_fuel_price, tfp, **params):
        """Non-renewable energy sector demand for fossil fuels."""
        return (tfp * (1 - beta) * (energy_price / fossil_fuel_price))**(1 / beta)
    

## Market for energy

In [ ]:
class EnergyMarket:
    
    def __init__(self, consumer, renewable_sector, non_renewable_sector):
        self.consumer = consumer
        self.renewable_sector = renewable_sector
        self.non_renewable_sector = non_renewable_sector
        
    def price(self, non_renewable_sector_capital, renewable_sector_capital, renewable_sector_params):
        return (((self.consumer.demand() - self.renewable_sector.output(renewable_sector_capital, **renewable_sector_params)) / capital)**beta * (1 / psi))**(1 / (1 - beta)) * ((1 - beta) / fossil_fuel_price)
        

## Physical capital market

## Model parameters

In [17]:
renewable_sector_params = {'alpha': 0.3, 'tfp': 3.692, 'gross_interest_rate': 1.09}
non_renewable_sector_params = {'beta': 0.67, 'tfp': 1, 'fossil_fuel_price': 1, 'phi': 0.2, 'eta': 0.9, 'psi': 1}

In [27]:
RenewableEnergySector.capital_demand(energy_price = 1.0, **renewable_sector_params)

1.023146409367968

In [31]:
RenewableEnergySector.profits(1.0, energy_price=1.0, **renewable_sector_params)

2.6020000000000003

In [7]:
def non_renewable_capital_dynamics(capital, previous_capital, energy_market, renewable_params, non_renewable_params):
    """Second order difference equation describes the dynamics of capital stock."""
    energy_price = energy_market.price(renewable_params, non_renewable_params)
    fossil_fuel = non_renewable_sector.fossil_fuel_demand(energy_price, non_renewable_params)
    future_capital = energy_market.non_renewable_sector.future_capital_demand(capital, previous_capital, fossil_fuel, energy_price)
    return future_capital


In [ ]:
def simulate_non_renewable_capital_dynamics(energy_market, renewable_params, non_renewable_params):
    capital = np.empty(T)
    capital[0:2] = initial_capital
    
    for t in range(T):
        capital[t] = non_renewable_capital_dynamics(capital[t-1], capital[t-2], energy_market, renewable_params, non_renewable_params)
    
    return capital